# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.



In [54]:
# Import dependencies
from pymongo import MongoClient
from pprint import pp
import json

Import the dataset with 
```bash
mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json
```

In [55]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)


In [56]:
# confirm that our new database was created
print(mongo.list_database_names())


['admin', 'autosaurus', 'classDB', 'config', 'epa', 'establishments', 'fruits_db', 'local', 'met', 'petsitly_marketing', 'travel_db', 'uk_food']


In [57]:
# assign the uk_food database to a variable name
uk_food = mongo['uk_food']

In [58]:
# review the collections in our new database
print(uk_food.list_collection_names())


['establishments']


In [59]:
# review a document in the establishments collection
establishments_docs = uk_food.establishments.find_one()

# Print one document
pp(establishments_docs)

{'_id': ObjectId('66a869e04c4875b3d700a5e8'),
 'FHRSID': 1290215,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000083029',
 'BusinessName': 'FirstLight Bar & Café',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': '1 The Droveway',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'PostCode': 'CT15 6DH',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2020-11-09T00:00:00',
 'LocalAuthorityCode': '182',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.378231', 'latitude': '51.150625'},
 'RightToReply': '',
 'Distance': 4587.64041272672,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T00:00:00',
          'i

In [60]:
# assign the collection to a variable
establishments = uk_food['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [61]:
# Create a dictionary for the new restaurant data
uk_food.establishments.insert_one(
    {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
    }       
)

InsertOneResult(ObjectId('66a86a38affd1eea48935cd3'), acknowledged=True)

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [62]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
fields = {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}  # _id:0 excludes the _id field

# Find the document matching the query
result = establishments.find_one(query, fields)

# Print the result
print(result)

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [63]:
# Update the new restaurant with the correct BusinessTypeID
# Define the filter to find the restaurant
filter = {"BusinessName": "Penang Flavours"}

# Define the update operation to change the BusinessTypeID
new_business_type_id = 1  # update with the new BusinessTypeID
update = {"$set": {"BusinessTypeID": new_business_type_id}}

# Update the document
update_result = establishments.update_one(filter, update)

In [64]:
# Confirm that the new restaurant was updated
if update_result.matched_count > 0:
    print(f"Updated BusinessTypeID for {update_result.modified_count} restaurant(s).")
else:
    print("No matching restaurant found.")
    
#count total documents
total_count = establishments.count_documents({})

# Print the total count using pprint
print(f"Total number of documents in the collection: {total_count}")

Updated BusinessTypeID for 1 restaurant(s).
Total number of documents in the collection: 39780


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [65]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {"LocalAuthorityName": "Dover"}
print("Number of documents in result (Dover Local Authority):", establishments.count_documents(query))

Number of documents in result (Dover Local Authority): 994


In [66]:
# Delete all documents where LocalAuthorityName is "Dover"
uk_food.establishments.delete_many(
    {"LocalAuthorityName": "Dover"}
)


DeleteResult({'n': 994, 'ok': 1.0}, acknowledged=True)

In [67]:
# Check if any remaining documents include Dover
query = {"LocalAuthorityName": "Dover"}
print("Number of documents in result (Dover Local Authority):", establishments.count_documents(query))

Number of documents in result (Dover Local Authority): 0


In [68]:
# Check that other documents remain with 'find_one'
establishments_docs = uk_food.establishments.find_one()

# Print one document
pp(establishments_docs)

{'_id': ObjectId('66a869e04c4875b3d700a8cd'),
 'FHRSID': 1043695,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': 'CT19 6BL',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2018-04-04T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.195625', 'latitude': '51.083812'},
 'RightToReply': '',
 'Distance': 4591.765489457773,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
          'extractDate': '00

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [69]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many(
    {},  # Empty filter to update all documents
    [
        {
            '$set': {
                "geocode.longitude": {
                    '$convert': {
                        'input': "$geocode.longitude",
                        'to': "decimal",
                        'onError': None,
                        'onNull': None
                    }
                },
                "geocode.latitude": {
                    '$convert': {
                        'input': "$geocode.latitude",
                        'to': "decimal",
                        'onError': None,
                        'onNull': None
                    }
                }
            }
        }
    ]
)


UpdateResult({'n': 38786, 'nModified': 38786, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

Use `update_many` to convert `RatingValue` to integer numbers.

In [70]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [71]:
# Change the data type from String to Integer for RatingValue
establishments.update_many(
    {},  # Empty filter to update all documents
    [
        {
            '$set': {
                "RatingValue": {'$convert': {'input': "$RatingValue", 'to': "int", 'onError': 0, 'onNull': 0}}
            }
        }
    ]
)

UpdateResult({'n': 38786, 'nModified': 38786, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [72]:
# Check that the coordinates and rating value are now numbers
# Retrieve a sample of documents to check the data type of RatingValue
sample_documents = establishments.find({}, {"RatingValue": 1, "geocode": 1}).limit(5)
# Limiting to 5 documents

# Check and print the data type of RatingValue for each document
for doc in sample_documents:
    rating_value = doc.get("RatingValue")
    longitude = doc.get("geocode", {}).get("longitude")
    latitude = doc.get("geocode", {}).get("latitude")
    print(f"Document ID: {doc['_id']}")
    print(f"  RatingValue: {rating_value}, Type: {type(rating_value)}")
    print(f"  Longitude: {longitude}, Type: {type(longitude)}")
    print(f"  Latitude: {latitude}, Type: {type(latitude)}")

Document ID: 66a869e04c4875b3d700a8cd
  RatingValue: 5, Type: <class 'int'>
  Longitude: 1.195625, Type: <class 'bson.decimal128.Decimal128'>
  Latitude: 51.083812, Type: <class 'bson.decimal128.Decimal128'>
Document ID: 66a869e04c4875b3d700a8ce
  RatingValue: 4, Type: <class 'int'>
  Longitude: 1.196408, Type: <class 'bson.decimal128.Decimal128'>
  Latitude: 51.086058, Type: <class 'bson.decimal128.Decimal128'>
Document ID: 66a869e04c4875b3d700a8cf
  RatingValue: 5, Type: <class 'int'>
  Longitude: 1.194762, Type: <class 'bson.decimal128.Decimal128'>
  Latitude: 51.085797, Type: <class 'bson.decimal128.Decimal128'>
Document ID: 66a869e04c4875b3d700a8d2
  RatingValue: 5, Type: <class 'int'>
  Longitude: 1.188537, Type: <class 'bson.decimal128.Decimal128'>
  Latitude: 51.08084, Type: <class 'bson.decimal128.Decimal128'>
Document ID: 66a869e04c4875b3d700a8d3
  RatingValue: 5, Type: <class 'int'>
  Longitude: 1.188537, Type: <class 'bson.decimal128.Decimal128'>
  Latitude: 51.08084, Type: